In [3]:
import os
os.chdir('/home/jupyter/gitlab/advertorial-classifier/')

In [4]:
from advertorial import dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
#import wandb
import numpy as np
#import evaluate

In [5]:
# advertorial_dataset = dataset.train_valid_test_from_file(csv_file_path= './data/milelens_advertorial_dataset_formatted.csv')
# train, validation, test = advertorial_dataset['train'], advertorial_dataset['validation'], advertorial_dataset['test'] 
# id2label = {0: "no", 1: "yes"}
# label2id = {"no": 0, "yes": 1}

# pretrain_model ="hfl/chinese-bert-wwm-ext"
# tokenizer = AutoTokenizer.from_pretrained(pretrain_model)
# model = AutoModelForSequenceClassification.from_pretrained(
#     pretrain_model, num_labels=2, id2label=id2label, label2id=label2id)



advertorial_dataset = dataset.train_valid_test_from_file(csv_file_path='./data/advertorial_test_from_pd_2023-05-31.csv', train_ratio=1 )#'./data/unseen_2023-05-30.csv', train_ratio=1)
train = advertorial_dataset['train']
id2label = {0: "no", 1: "yes"}
label2id = {"no": 0, "yes": 1}

pretrain_model ="hfl/chinese-bert-wwm-ext"
tokenizer = AutoTokenizer.from_pretrained(pretrain_model)
model = AutoModelForSequenceClassification.from_pretrained(
    pretrain_model, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

In [18]:
train_dataset = dataset.train_valid_test_from_file(csv_file_path='./data/train_set_2023-05-31.csv', train_ratio=1 )
valid_dataset = dataset.train_valid_test_from_file(csv_file_path='./data/valid_set_2023-05-31.csv', train_ratio=1 )
#'./data/unseen_2023-05-30.csv', train_ratio=1)
train = train_dataset['train']
valid = valid_dataset['train']
id2label = {0: "no", 1: "yes"}
label2id = {"no": 0, "yes": 1}

pretrain_model ="hfl/chinese-bert-wwm-ext"
tokenizer = AutoTokenizer.from_pretrained(pretrain_model)
model = AutoModelForSequenceClassification.from_pretrained(
    pretrain_model, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

In [19]:
from advertorial.inference import AdvertorialModel
adv = AdvertorialModel(model_path='./prebuilt_model/230531_chinese_bert_wwm_ext', use_gpu=True)

In [20]:
import pandas as pd
def perf_report(model, dataset, name='train'):
    from tqdm import tqdm
    N = len(dataset)
    step = 20
    ones = 0
    zeros = 0
    hits = 0
    miss = 0
    predictions = []
    for s in tqdm(range(0, N, step)):
        s, e = s, s+step
        prediction, probs = model(dataset[s:e]['text'])
        
        hits += np.sum(dataset[s:e]['label'] == prediction)
        miss += np.sum(dataset[s:e]['label'] != prediction)
        zeros += np.sum(dataset[s:e]['label'] == np.array(0))
        ones += np.sum(dataset[s:e]['label'] == np.array(1))
        predictions.append(prediction) 

    accuracy = hits/N
    print(f'accuracy:{accuracy:.2f}, positive samples:{ones}, negative samples:{zeros}')  
    performance_df = pd.DataFrame({'dataset':[name], 
                                   'records':[N], 
                                   'positive samples':[ones], 
                                   'negative samples':[zeros], 
                                   'hit':[hits],
                                   'miss':[miss],
                                   'accuracy':[accuracy], 'miss rate':[1-accuracy]})

    predictions = np.concatenate(predictions)
    error_ids = predictions != dataset['label']
    error_df = pd.DataFrame({'text':np.array(dataset['text'])[error_ids], 'label':np.array(dataset['label'])[error_ids], 'prediction':predictions[error_ids]})
    return error_df, performance_df

In [21]:
train_error, train_perf = perf_report(adv, train, 'train')
validation_error, validation_perf = perf_report(adv, valid, 'validation')
#test_error, test_perf = perf_report(adv, test, 'test')

100%|██████████| 946/946 [05:35<00:00,  2.82it/s]


accuracy:0.96, positive samples:7300, negative samples:11607


100%|██████████| 237/237 [01:24<00:00,  2.82it/s]

accuracy:0.96, positive samples:1892, negative samples:2835


In [22]:
#pd.concat([train_perf, validation_perf, test_perf]).reset_index(drop=True)#.to_csv('performance.csv', index=False)
pd.concat([train_perf, validation_perf]).reset_index(drop=True)#.to_csv('performance.csv', index=False)

,dataset,records,positive samples,negative samples,hit,miss,accuracy,miss rate
0,train,18907,7300,11607,18151,756,0.960015,0.039985
1,validation,4727,1892,2835,4533,194,0.958959,0.041041


In [13]:
df = pd.read_excel('./data/業配文期末考題目.xlsx', sheet_name='Sheet1')
#df = df.rename(columns={'貼文內容':'text', '正確答案':'label'})[['text', 'label']]

In [17]:
join = train_error.merge(df, left_on='text', right_on='貼文內容', how='left')
join[['貼文內容', '正確答案', 'label', 'prediction', '貼文連結']].to_csv('業配文期末考題目_錯誤部分.csv', index=False)